In [1]:
!mkdir -p /tmp/pip/cache/
!cp ../input/steel-my-models/efficientnet_pytorch-0.5.1.xyz /tmp/pip/cache/efficientnet_pytorch-0.5.1.tar.gz

In [2]:
!pip install --no-index --find-links /tmp/pip/cache/ efficientnet-pytorch

Looking in links: /tmp/pip/cache/
Processing /tmp/pip/cache/efficientnet_pytorch-0.5.1.tar.gz
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.5.1-cp36-none-any.whl size=11768 sha256=ca1f4571c326720edc339eab9962bff557de77ea43bb39697a58ae63ccd1b6d3
  Stored in directory: /tmp/.cache/pip/wheels/5c/5a/43/cd0c920c44f367c447b35c79f795910683cb26cd51579b328f
Successfully built efficientnet-pytorch


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import time

import albumentations as albu
from albumentations.pytorch import ToTensor
import PIL
import cv2 as cv

import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms.functional as TF
from torch import nn
import torch.nn.functional as F
from torch.utils.data import SubsetRandomSampler
from torch.optim import Adam,lr_scheduler

from efficientnet_pytorch import EfficientNet
from efficientnet_pytorch.utils import Conv2dStaticSamePadding, get_model_params

from tqdm import tqdm_notebook, tqdm

In [4]:
# setup the input data folder
DATA_PATH = '../input/bengaliai-cv19/'

# load the dataframes with labels
train_labels = pd.read_csv(DATA_PATH + 'train.csv')
test_labels = pd.read_csv(DATA_PATH + 'test.csv')
class_map = pd.read_csv(DATA_PATH + 'class_map.csv')
sample_submission = pd.read_csv(DATA_PATH + 'sample_submission.csv')

In [5]:
def threshold_image(img):
    '''
    Helper function for thresholding the images
    '''
    gray = PIL.Image.fromarray(np.uint8(img), 'L')
    ret,th = cv.threshold(np.array(gray),0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    return th

In [6]:
efficientnet_b0 = EfficientNet.from_name('efficientnet-b0')

In [7]:
class BengaliModel(nn.Module):
    def __init__(self, backbone_model):
        super(BengaliModel, self).__init__()
        self.conv = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3)
        self.backbone_model = backbone_model
        self.fc1 = nn.Linear(in_features=1000, out_features=168) # grapheme_root
        self.fc2 = nn.Linear(in_features=1000, out_features=11) # vowel_diacritic
        self.fc3 = nn.Linear(in_features=1000, out_features=7) # consonant_diacritic
        
    def forward(self, x):
        # pass through the backbone model
        y = self.conv(x)
        y = self.backbone_model(y)
        
        # multi-output
        grapheme_root = self.fc1(y)
        vowel_diacritic = self.fc2(y)
        consonant_diacritic = self.fc3(y)
        
        return grapheme_root, vowel_diacritic, consonant_diacritic

In [8]:
model = BengaliModel(efficientnet_b0)

In [9]:
# setup training device
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [10]:
#train the model
model = model.to(device)

In [11]:
state = torch.load('../input/bengaliaiutils/efficientnet_b0_10.pth', map_location=lambda storage, loc: storage)
model.load_state_dict(state["state_dict"])

<All keys matched successfully>

In [12]:
def get_predicted_label(ps):
    ps = F.softmax(ps)
    top_p, top_class = ps.topk(1, dim=1)
        
    top_p = top_p.detach().numpy()
    top_class = top_class.detach().numpy()
    
    return np.array(top_class).reshape(ps.shape[0])

In [13]:
def do_predict(model, imgs):
    imgs = imgs.to(device)
    grapheme_root, vowel_diacritic, consonant_diacritic  = model.forward(imgs)
    imgs = imgs.cpu()
    grapheme_root = grapheme_root.cpu()
    vowel_diacritic = vowel_diacritic.cpu()
    consonant_diacritic = consonant_diacritic.cpu()
    
    grapheme_root_labels = get_predicted_label(grapheme_root)
    vowel_diacritic_labels = get_predicted_label(vowel_diacritic)
    consonant_diacritic_labels = get_predicted_label(consonant_diacritic)
    
    return grapheme_root_labels, vowel_diacritic_labels, consonant_diacritic_labels

In [14]:
submission = pd.DataFrame(columns=['row_id', 'target'])

In [15]:
def run_make_submission_csv(submission):
    row_id=[]
    target=[]
    batch_size= 128

    for i in range(4):
        df  = pd.read_parquet(DATA_PATH+'/test_image_data_%d.parquet'%i, engine='pyarrow')
        #df  = pd.read_parquet(DATA_PATH+'/train_image_data_%d.parquet'%i, engine='pyarrow') #use this to test timing
        
        num_test = len(df)
        for b in range(0,num_test,batch_size):
            
            since = time.time()
            
            B = min(num_test,b+batch_size)-b
            image = df.iloc[b:b+B, range(1,32332+1)].values
            image_ids = df.iloc[b:b+B, 0].values

            image = image.reshape(B,1,137, 236).astype(float)
            # preprocess the image
            for j in range(B):
                image[j,:,:] = threshold_image(image[j,:,:].reshape(137, 236)).reshape(1,1,137, 236)

            input = torch.from_numpy(image).float().cuda()
            grapheme_root_labels, vowel_diacritic_labels, consonant_diacritic_labels = do_predict(model, input)

            for ind in range(input.shape[0]):
                image_id = image_ids[ind]

                grapheme_root_label = grapheme_root_labels[ind]
                vowel_diacritic_label = vowel_diacritic_labels[ind]
                consonant_diacritic_label = consonant_diacritic_labels[ind]

                submission = submission.append({'row_id':str(image_id)+'_grapheme_root', 'target':grapheme_root_label}, 
                                               ignore_index=True)
                submission = submission.append({'row_id':str(image_id)+'_vowel_diacritic', 'target':vowel_diacritic_label}, 
                                               ignore_index=True)
                submission = submission.append({'row_id':str(image_id)+'_consonant_diacritic', 'target':consonant_diacritic_label}, 
                                               ignore_index=True)
                
            print('Time elapsed:{}'.format(time.time() - since))
            break;
        break;
    return submission

In [16]:
submission = run_make_submission_csv(submission)

Time elapsed:0.7342050075531006


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [17]:
submission.head()

,row_id,target
0,Test_0_grapheme_root,64
1,Test_0_vowel_diacritic,1
2,Test_0_consonant_diacritic,0
3,Test_1_grapheme_root,93
4,Test_1_vowel_diacritic,2


In [18]:
submission.to_csv('submission.csv', index=False)